---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
     
    
    UniTownsList = [x.strip() for x in open('university_towns.txt')]
    UniTowns = pd.DataFrame({0:UniTownsList})
    
    UniTowns['State'] = UniTowns[0].apply(lambda x: x.split('[ed')[0].strip(
    ) if x.count('[ed') > 0 else np.NaN).fillna(method="ffill")
    
    UniTowns['RegionName'] = UniTowns[0].apply(lambda x: x.split(
    '(')[0].strip() if x.count('(')>0 else x)
    
    UniTowns['RegionName'] = UniTowns['RegionName'].apply(lambda x: x.strip() if x.count('[ed')==0 else np.NaN)


    UniTowns['State'] = UniTowns[0].apply(lambda x: x.split('[ed')[0].strip(
    ) if x.count('[ed') > 0 else np.NaN).fillna(method="ffill")

    UniTowns = UniTowns.dropna().drop(0,axis=1).reindex(
    columns=['State', 'RegionName'])
    
    return UniTowns

get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls',header=5,index_col=None,usecols=[4,5,6],
                       names=['Quarters','GDP','ChainedGDP'],skiprows=214)
    df=df.set_index('Quarters')
    df.index.name='Quarters'
    #df=df.diff(periods=1,axis=0)
    #df.loc['2000q1','ChainedGDP']=0
    #Want last month of growth before two months of decline
    df['RecStart']=np.where(df['ChainedGDP']<0,1,0)
    answer=0
    for i in range(0, len(df)):
        first_q=df.iloc[i]['GDP']
        second_q=df.iloc[i+1]['GDP']
        third_q=df.iloc[i+2]['GDP']
        fourth_q=df.iloc[i+3]['GDP']
        if (first_q<second_q)&((third_q<second_q)&(fourth_q<third_q)):
            answer=df.iloc[i+1]
            
            break
    
            
    return answer.name
get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls',header=5,index_col=None,usecols=[4,5,6],
                       names=['Quarters','GDP','ChainedGDP'],skiprows=214)
    df=df.set_index('Quarters')
    #Want last month of growth before two months of decline
   
    answer=0
    start=0
    for i in range(0, len(df)):
        first_q=df.iloc[i]['GDP']
        second_q=df.iloc[i+1]['GDP']
        third_q=df.iloc[i+2]['GDP']
        fourth_q=df.iloc[i+3]['GDP']
        if (first_q<second_q)&((third_q<second_q)&(fourth_q<third_q)):
            start=i+1
            break
    for i in range(0, len(df)-start):
        first_q=df.iloc[start+i]['GDP']
        second_q=df.iloc[start+i+1]['GDP']
        third_q=df.iloc[start+i+2]['GDP']
        fourth_q=df.iloc[start+i+3]['GDP']
        if (first_q>second_q)&((third_q>second_q)&(fourth_q>third_q)):
            answer=df.iloc[start+i+3]
            
            break
       
    return answer.name
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls',header=5,index_col=None,usecols=[4,5,6],
                       names=['Quarters','GDP','ChainedGDP'],skiprows=214)
    df=df.set_index('Quarters')
    #Want last month of growth before two months of decline
   
    answer=0
    start=0
    for i in range(0, len(df)):
        first_q=df.iloc[i]['GDP']
        second_q=df.iloc[i+1]['GDP']
        third_q=df.iloc[i+2]['GDP']
        fourth_q=df.iloc[i+3]['GDP']
        if (first_q<second_q)&((third_q<second_q)&(fourth_q<third_q)):
            start=i+1
            break
    for i in range(0, len(df)-start):
        first_q=df.iloc[start+i]['GDP']
        second_q=df.iloc[start+i+1]['GDP']
        third_q=df.iloc[start+i+2]['GDP']
        fourth_q=df.iloc[start+i+3]['GDP']
        if (first_q>second_q)&((third_q>second_q)&(fourth_q>third_q)):
            answer=df.iloc[start+i+1]
            
            break
                    
            
    return answer.name
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    Housing=pd.read_csv('City_Zhvi_AllHomes.csv')
    
    Housing['State'].replace(states,inplace=True)
    Housing = Housing.set_index(['State','RegionName'])
    THousing=Housing.iloc[:,51:]
    THousing.columns=pd.to_datetime(THousing.columns)
    THousing = THousing.resample('Q',axis=1).mean()
    THousing = THousing.rename(columns=lambda x: str(x.to_period('Q')).lower())
    
    return THousing
convert_housing_data_to_quarters()

In [ ]:
from scipy import stats
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    Housing = convert_housing_data_to_quarters()
    start = get_recession_start()
    bottom = get_recession_bottom()
    UniList=get_list_of_university_towns()
    Unidf = pd.DataFrame(UniList)
    Unidf['UniFlag']=1
    
    Housing['Ratio']=Housing[str(start)]/Housing[str(bottom)]
    MHousing=pd.merge(Housing,Unidf,how='left',left_index=True,right_on=['State','RegionName'])
    MHousing = MHousing.dropna(subset=['Ratio'])
    UniRatio = MHousing[MHousing['UniFlag']==1]
    #Not sure how we get everyone outside - maybe easier to merge on a flag if in UniList?
    NonUniRatio = MHousing[pd.isnull(MHousing['UniFlag']) ]
    
    p=stats.ttest_ind(UniRatio['Ratio'],NonUniRatio['Ratio'])[1]
    different = (p<.01)
    if UniRatio['Ratio'].mean()<NonUniRatio['Ratio'].mean(): better = "university town"
    else: better = "non-university town"
    #need to drop NAN    
    
    return (different,p,better)
run_ttest()